## Datapane Tutorial - Google Trends Analysis

---

This is part of a series of tutorials to help you learn how to use Datapane.

Before getting started, you will need your API token, which you can find on your [settings page](https://datapane.com/settings). If you are on a private Datapane instance, you will find it in `/settings` on your instance domain (i.e., https://[your-instance].datapane.net/settings).

Once you have you token, add it to the form field in the cell below to login to Datapane.

If you are using Datapane Community, set the `datapane_server_url` as `https://datapane.com`. Alternatively, if you are using a Teams instance, enter the URL of your instance (i.e. https://[your-instance].datapane.net).


In [ ]:
datapane_api_token = "" #@param {type:"string"}
datapane_server_url = "https://datapane.com" #@param {type:"string"}

In [ ]:
!pip3 install --quiet datapane pytrends

import datapane as dp
dp.login(token=datapane_api_token, server=datapane_server_url) # Make sure you've set your API token above!

# Import libraries and set parameters

Although we can generate our Datapane report by manually running this notebook on Colab or our local machine, in the future we may want to deploy it to Datapane Cloud as an app which stakeholders can run with parameters.

To enable this, we are using `dp.Params`, which allows us to access parameters when this notebook is run as a Datapane App (where Datapane will use our parameters to generate a form for end-users).

Params is a Python dictionary which is injected at runtime and will be empty if we are running it elsewhere, so we are choosing some defaults.


In [ ]:
import pandas as pd
from pytrends.request import TrendReq
import altair as alt
import datapane as dp
pytrends = TrendReq(tz=360)

# Choose your query!
keyword = dp.Params.get('keyword', "python")
start_date = dp.Params.get('timeframe', '3-m')
pytrends.build_payload([keyword], cat=0, timeframe=f'today {start_date}', geo='', gprop='')
top_queries = pytrends.related_queries()[keyword]['top']
related = top_queries.style.bar(subset=['value'], align='mid', color=['#d65f5f', '#5fba7d'])

## Plotting our trends

In [ ]:
import altair as alt
alt.themes.enable('quartz')
fig = alt.Chart(top_queries).mark_bar().encode(x='query', y='value').interactive()
fig

## Interest over Time

In [ ]:
r_o_t = pytrends.interest_over_time().reset_index()
r_o_t.rename(columns={keyword : 'search_volume'}, inplace=True)
r_o_t_plot = alt.Chart(r_o_t).encode(x='date', y='search_volume').mark_area(line=True).interactive()
r_o_t_plot

## Geographical Interest

In [ ]:
regions = pytrends.interest_by_region(resolution='COUNTRY', inc_geo_code=True)
regions.rename(columns={keyword : 'search_volume'}, inplace=True)
regions.reset_index()

subset = regions.reset_index().sort_values(by='search_volume', ascending=False)[:20]
bar = alt.Chart(subset).encode(x='geoName', y='search_volume').mark_bar()
bar

In [ ]:
from datetime import date

heading = f"""<div style="display:flex;justify-content:center;background: rgb(229, 231, 235);flex-direction: column;text-align: center;margin-bottom: ;">
  <h1 style="font-size: 2em;">SEO forecast analysis of <code style="color: rgb(129, 140, 248); font-size:1.2em;">{keyword}</code></h1>
</div>"""

dp.Report(
    dp.Page(blocks=[
        dp.HTML(heading),
"This data was pulled from the Google Trends API for your search term. This report is refreshed _daily_, but please reach out if you need to change the cadence.",
"## Search volume",
        dp.Plot(r_o_t_plot, label='Observed data')
    ], title='Interest over time'),
    dp.Page(blocks=[
        "These are queries which are related to your keyword",
        fig,
    ], title='Related queries'),
    dp.Page(blocks=[
        "This data was pulled from the Google Trends API",
        dp.DataTable(r_o_t, caption='Time series data')
    ], title='Input Dataset')
).upload(name="Trends Report")
